In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts

import matplotlib.pyplot as plt
import pylab

import os 

In [2]:
defen = pd.read_excel('Data_Extract_From_Gender_Statistics.xlsx', sheetname=1, encoding = 'utf8').append(pd.read_excel('Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheetname=1, encoding='utf8')).append(pd.read_excel('Data_Extract_From_Millennium_Development_Goals.xlsx', sheetname=1, encoding='utf8'))

In [15]:
def D_maker(data,c=0.5, p=0.01, m=1, s=u'Корреляция'):
    rez=[]
    for index in data.index:
        for column in data.columns:
            i_tmp = index.split(':')
            c_tmp = column.split(':')
            value=[]
            
            if i_tmp[2]!=u'p-value' and i_tmp[0]!=c_tmp[0]:
                value.append([data.get_value(index=index, col=column),data.get_value(index=i_tmp[0]+':'+i_tmp[1]+':p-value', col=column)])
                
                if abs(value[0][0])>c and value[0][1]<p and abs(value[0][0])<m:
                    
                    if [defen[defen['Code']==c_tmp[0]]['Indicator Name'].tolist()[0], defen[defen['Code']==i_tmp[0]]['Indicator Name'].tolist()[0], value[0][0]] not in rez:
                        rez.append([defen[defen['Code']==i_tmp[0]]['Indicator Name'].tolist()[0], defen[defen['Code']==c_tmp[0]]['Indicator Name'].tolist()[0], value[0][0]])
                    
                    #Degbug                    
                    #return rez
    db = pd.DataFrame().append(rez)
    db.columns = [u'Первый признак', u'Второй признак', u'Корреляция']
    #print (len(db))
    db = check(db.sort_values(by=[s]))
    return db

def check(data):
    rez=[]
    data = data.sort_values(by=[u'Корреляция'])
    tmp = data.as_matrix()
    for i, row in enumerate(tmp):
        if i<len(tmp)-1:
            if row[2] != tmp[i+1][2] and row[1] != tmp[i+1][0] and row[0] != tmp[i+1][1]:
                rez.append(row)
    if len(rez)!=0:
        db = pd.DataFrame().append(rez)
        db.columns = [u'Первый признак', u'Второй признак', u'Корреляция']
        return db.reset_index()
    else:
        print('what?')
        return data
            

def to_ex(data, name, n_region):
    filename = name+'.'+n_region+'.Con.xlsx'
    data[[u'Первый признак', u'Второй признак', u'Корреляция']].sort_values(by=u'Первый признак').to_excel(filename, sheet_name=name, index = False)

def Sumup(d_name):
    files = os.listdir(d_name)
    rez = pd.DataFrame(columns = [u'Первый признак', u'Второй признак', u'Корреляция'])
    for name in files:
        data = pd.read_excel(d_name+'/'+name ,encoding = 'utf8')
        n_region = name.split('.')[1].split('_')[2]
        print (n_region)
        to_ex(D_maker(data), name.split('.')[0], n_region)

In [23]:
to_ex(D_maker(pd.read_excel('deep.Corr_in_RUS.xlsx', encoding='utf8')), 'check', 'RUS')